In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
TPU_INIT = False

if TPU_INIT:
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
        tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
    except ValueError:
        raise BaseException('ERROR: Not connected to a TPU runtime!')


print("Tensorflow version " + tf.__version__)

In [ ]:
# Tinggi dan lebar untuk mengubah ukuran frame video
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
# Jumlah frame yang akan diambil dari setiap video
SEQUENCE_LENGTH = 16
# Direktori tempat dataset disimpan
DATASET_DIR = "../data/"
# Daftar class untuk klasifikasi
CLASSES_LIST = ["NonViolence", "Violence"]

In [ ]:
def extract_frames(video_path):

    cap = cv2.VideoCapture(video_path)

    # 1. Dapatkan total frame
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 2. Tentukan indeks frame yang akan diambil, sampling merata
    if total_frames < SEQUENCE_LENGTH:
        # Kalau video terlalu pendek, ambil semua dan ulangi terakhir sampai seq_len
        indices = list(range(total_frames)) + [total_frames - 1] * (SEQUENCE_LENGTH - total_frames)
    else:
        step = total_frames / SEQUENCE_LENGTH
        indices = [int(i * step) for i in range(SEQUENCE_LENGTH)]

    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            # kalau gagal baca frame, ulangi frame terakhir
            frame = frames[-1] if frames else np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 3), np.uint8)
        # 3. Center‐crop ke square
        h, w = frame.shape[:2]
        m = min(h, w)
        y0 = (h - m) // 2
        x0 = (w - m) // 2
        crop = frame[y0:y0+m, x0:x0+m]
        # 4. Resize, RGB, normalisasi
        crop = cv2.resize(crop, (IMAGE_HEIGHT, IMAGE_WIDTH), interpolation=cv2.INTER_AREA)
        rgb  = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        frames.append(rgb.astype(np.float32) / 255.0)

    cap.release()
    return np.stack(frames, axis=0)

In [ ]:
def create_dataset():
    features = []
    labels = []
    video_files_paths = []

    # 1. Iterasi melalui setiap kelas
    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class: {class_name}')
        
        # 2. Dapatkan daftar file video dalam direktori kelas
        class_dir = os.path.join(DATASET_DIR, class_name)
        files_list = os.listdir(class_dir)

        # 3. Iterasi melalui setiap file video
        for file_name in files_list:
            # a. Bangun path lengkap ke file video
            video_file_path = os.path.join(class_dir, file_name)

            # b. Ekstrak frame dari video
            frames = extract_frames(video_file_path)

            # c. Periksa apakah jumlah frame sesuai dengan SEQUENCE_LENGTH
            if len(frames) == SEQUENCE_LENGTH:
                # d. Simpan data ke dalam list yang sesuai
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    # 4. Konversi list ke array NumPy
    features = np.asarray(features)
    labels = np.array(labels)

    # 5. Kembalikan dataset
    return features, labels, video_files_paths


In [ ]:
# Buat dataset
features, labels, video_files_paths = create_dataset()

In [ ]:
# Konversi label ke one hot encoded format
one_hot_encoded_labels = to_categorical(labels)

In [ ]:
# Split data menjadi 90% data train dan 10% data test.
X_train, X_test, y_train, y_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.1, shuffle = True, random_state = 42)


In [ ]:
print(X_train.shape,y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# Saving the extracted data
os.makedirs("../data/ProcessedData/", exist_ok=True)
np.save("../data/ProcessedData/X_train.npy", X_train)
np.save("../data/ProcessedData/y_train.npy", y_train)
np.save("../data/ProcessedData/X_test.npy", X_test)
np.save("../data/ProcessedData/y_test.npy", y_test)
print("Data has been saved successfully.")